In [1]:
# !pip install langchain
# !pip install langchain-openai

In [2]:
from langchain_openai import OpenAIEmbeddings

In [3]:
# !pip install langchain
# !pip install openai
# !pip install PyPDF2
# !pip install faiss-cpu
# !pip install tiktoken
# !pip install typing-inspect==0.8.0
# !pip install typing_extensions==4.5.0
# !pip install pydantic==1.10.11

In [4]:
from PyPDF2 import PdfReader
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [5]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-gZn62Lvf7o8AL1p78N0ET3BlbkFJrZlxbvq060o7TBPqoGFP"

In [6]:
pdfreader = PdfReader('Anubhav Hooda Resume.pdf')
# job_reader = PdfReader(job_description.)

In [7]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [8]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)
cleaned_texts = [chunk.replace("\n", " ") for chunk in texts]
cleaned_texts = [chunk.replace("_", "") for chunk in cleaned_texts ]
# cleaned_texts = re.sub('_{2,}', ' ', cleaned_texts)
print(cleaned_texts)

['Anubhav Hooda 1 Anubhav Hooda Web3 Developer and Blockchain Connoisseur  Advanced Machine Learning Skills , CyberSec Enthusiast Jenkins and Ansible  Contact +91 9911865100 hoodaanubhav@gmail.comNew Delhi, India https://www .linkedin.com/in/anubhavera/ Professional Aspect I have worked on WEB3 projects in the past using Solidity . I like to keep myself updated   on the fundamentals of Crypto coins and Blockchain . Recently full throttled into Machine Learning and Created the Depression detection   model that works on S-MRI and f-MRI scans to detect the absolute ef fects and parts of   brain af fected due to depression.  Developed the prototype of Robotic Arm controlled autonomously by Reinforcement   Learning ranging from Hardware designs in SolidW orks to Software simulations in', 'brain af fected due to depression.  Developed the prototype of Robotic Arm controlled autonomously by Reinforcement   Learning ranging from Hardware designs in SolidW orks to Software simulations in   Mujo

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [10]:
document_search = FAISS.from_texts(cleaned_texts, embeddings)

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [12]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

C:\Users\hooda\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [13]:
query = "What are the skills?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\hooda\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The skills listed are Web3 (75%), Blockchain (85%, Personal), Machine Learning (80%), Python (90%, Language), Ansible (75%, Technical), Jenkins (90%, Technical), HTML & CSS (90%, Technical), Solidity (40%, Language), JavaScript (70%, Technical), Linux (75%, Technical), and Microsoft Office (90%, Technical).'

In [14]:
query = "What are the projects?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The projects mentioned are: \n1) Reinforcement Learning to Train Robotic Arm \n2) Depression Detection Using ML trained on f-MRI scans \n3) Web3 Marketplace.'

In [15]:
query = "You are a seasoned recruiter tasked with evaluating the suitability of a candidate for the position of Back End Software Engineer based on their resume. Your assessment should be thorough and insightful, reflecting both the candidate's qualifications and the specific requirements outlined in the job description. Consider the candidate's technical skills, experience, and potential cultural fit within the organization. Provide a detailed analysis, including a numerical rating from 1 to 10 along with a comprehensive explanation justifying the score. Take into account both the strengths and areas for improvement highlighted in the resume and how well they align with the responsibilities and expectations outlined in the job description. Aim for a holistic assessment that goes beyond surface-level evaluation."
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

"\nBased on the candidate's resume, Anubhav Hooda appears to have a strong technical background and a passion for emerging technologies such as Blockchain and Machine Learning. However, there are also some areas for improvement that should be considered.\n\nIn terms of technical skills, Anubhav has a solid understanding of various technologies such as Python, Jenkins, Ansible, HTML & CSS, and Linux, with a strong proficiency in Python at 90%. The candidate also has some experience with Solidity and JavaScript, although their proficiency in these languages appears to be lower at 40% and 70%, respectively. Overall, the candidate's technical skills seem to align well with the requirements for a Back End Software Engineer position.\n\nIn terms of experience, Anubhav mentions working on WEB3 projects in the past using Solidity, which shows a practical application of their skills. Additionally, their work on a Depression Detection model using Machine Learning trained on f-MRI scans is impres

In [16]:
job_description = """
    The Back End Software Engineer is responsible for the following:
    • Develop server-side logic, definition and maintenance of the central database, and ensure high performance and responsiveness to requests from the front-end developers.
    • Integrate user-facing elements developed by front-end developers with server-side applications.
    • Collaborate with front-end developers, customers, users and Product Managers to establish objectives and design functional, cohesive codes to enhance the user experience.
    • Keep abreast of novel technical concepts and markets.
    • Provide technical leadership and documentation to developers and stakeholders.
    • Apply usability procedures and principles as defined at the project or Product Line level or through customer input.
    • Build prototypes, products and systems that meet the project quality standards and requirements.
    • Contribute to and support re-use through common components that are well documented and tested
    """

In [17]:
def analyze_resume_job_match(cleaned_texts, job_description):    
    query =  f"Resume:\n{cleaned_texts}\n\nJob Description:\n{job_description}\n\n Analyze how well the resume matches the job description, \
        then Analyze the resume and provide an estimate percentage of how well it matches the job description\
        Additionally, provide critiques on the resume:"
    docs = document_search.similarity_search(query)
    chain.run(input_documents=docs, question=query)

In [18]:
analyze_resume_job_match(cleaned_texts,job_description)